Data Source URL - https://networkrepository.com/soc-linkedin.php 

In [1]:
import pandas
import json
import os
from pandasql import sqldf

In [2]:
path = os.getcwd()

# Parse edges data

In [3]:
file_path2 = os.path.join(path, 'soc-linkedin.edges')
data2 = pandas.read_csv(file_path2, sep=' ', header=None, names = ['source', 'target'], dtype = {'source': str, 'target': str})

In [4]:
data2

,source,target
0,0,2152448
1,0,1656491
2,0,399364
3,0,18448
4,0,72025
...,...,...
19360685,6693303,6717406
19360686,6693599,6703600
19360687,6711680,6721437
19360688,6712636,6716154


In [5]:
counts = data2['source'].value_counts().rename_axis('unique_values').reset_index(name='counts')
counts

,unique_values,counts
0,0,869
1,1,702
2,2,608
3,3,564
4,4,532
...,...,...
3115264,4988272,1
3115265,3902148,1
3115266,1273553,1
3115267,2716393,1


In [16]:
between_25_and_10 = counts[(counts['counts']>= 10) & (counts['counts'] <= 25)]

In [17]:
between_25_and_10

,unique_values,counts
19009,24102,25
19010,24418,25
19011,18033,25
19012,16613,25
19013,19494,25
...,...,...
501286,780714,10
501287,917816,10
501288,1012281,10
501289,744210,10


In [38]:
filter = ['37409', '42856', '35278', '47492', '53931', '469301', '451872', '1185097', '456349', '347118',
         '151997',
'17881',
'7724',
'335',
'11331',
'19677',
'385916',
'156113',
'408921',
'20501',
'263828',
'23014',
'168814',
'20737',
'454968',
'63167']
# filter = ['35278'] #test

In [39]:
data3 = data2[data2.source.isin(filter)].copy()
data4 = data2[data2.target.isin(filter)].copy()
data3 = data3.append(data4)
data3 = data3.dropna(subset=['source', 'target']).reset_index(drop = True)
data3

,source,target
0,335,2672133
1,335,8714
2,335,1218572
3,335,1206800
4,335,83475
...,...,...
582,263828,451872
583,263828,385916
584,385916,451872
585,408921,469301


In [40]:
data3.source.value_counts()

335        103
7724        39
11331       31
20501       30
20737       28
19677       28
17881       28
23014       25
37409       20
35278       20
42856       20
47492       20
53931       20
63167       18
168814      16
263828      14
151997      13
156113      12
385916      12
454968      11
451872      10
469301      10
1185097     10
347118      10
456349      10
408921      10
24514        2
15798        2
26326        1
302          1
148678       1
62           1
77164        1
94806        1
112051       1
31368        1
10484        1
323          1
17300        1
169555       1
124226       1
116          1
31731        1
Name: source, dtype: int64

# Parse nodes data

In [21]:
file_path = os.path.join(path, 'soc-linkedin.nodes')
data = pandas.read_csv(file_path, sep = '\t',
                  header=None, error_bad_lines=False, index_col = None, names = ['id', 'username'], 
                       dtype = {'id': str, 'username': str})

In [22]:
data

,id,username
0,0,pub-sandra-arana-querol-2b-561-73
1,1,in-sehrishhafiz
2,2,pub-heba-bayoumi-16-a6-97
3,3,pub-aysha-binbraik-5-90b-b49
4,4,in-rubabadowla
...,...,...
6726285,6726285,pub-alejandra-paz-cadena-lucas-57-886-68b
6726286,6726286,pub-al-greenberg-1-336-bbb
6726287,6726287,pub-akira-kato-7-71a-525
6726288,6726288,pub-adi-shamir-a-b26-378


In [41]:
#Select top 100 users 
# data_subset = data[0:100].copy()
all_ids = pandas.unique(data3[['source', 'target']].values.ravel('K'))
data_subset = data[data['id'].isin(all_ids)].copy()
data_subset

,id,username
62,62,pub-chiraz-ben-ali-28-583-525
116,116,pub-jules-gough-30-407-744
302,302,pub-hanen-abid-23-a7-b91
323,323,pub-amira-ben-ghanem-23-770-b03
335,335,in-mahameddeb
...,...,...
5512131,5512131,pub-kung-lagula-2b-7ab-3b6
5666442,5666442,pub-jim-copley-9-339-73b
5907069,5907069,pub-fabio-pasolini-16-b0a-96b
6088895,6088895,pub-clyde-burgess-1a-7a3-626


In [42]:
len(data_subset)

456

In [43]:
data[data.id.isin(['35278'])]

,id,username
35278,35278,in-jcmonnet


### Merge nodes and edges files to get username source and target values

In [44]:
df = data3.merge(data, left_on='source', right_on='id', how='left')
df = df.rename(columns={"id": "source_id", "username": "source_username"})
df

,source,target,source_id,source_username
0,335,2672133,335,in-mahameddeb
1,335,8714,335,in-mahameddeb
2,335,1218572,335,in-mahameddeb
3,335,1206800,335,in-mahameddeb
4,335,83475,335,in-mahameddeb
...,...,...,...,...
582,263828,451872,263828,pub-rÃ©my-vadrot-24-913-818
583,263828,385916,263828,pub-rÃ©my-vadrot-24-913-818
584,385916,451872,385916,pub-pierre-breton-1-b86-362
585,408921,469301,408921,pub-marco-pesce-6-688-750


In [45]:
df2 = df.merge(data, left_on='target', right_on='id', how='left')
df2 = df2.rename(columns={"id": "target_id", "username": "target_username"})
df2

,source,target,source_id,source_username,target_id,target_username
0,335,2672133,335,in-mahameddeb,2672133,pub-hatem-bouattour-17-7aa-882
1,335,8714,335,in-mahameddeb,8714,pub-anja-slama-16-52a-868
2,335,1218572,335,in-mahameddeb,1218572,pub-amal-hammami-46-170-5a3
3,335,1206800,335,in-mahameddeb,1206800,pub-anis-jabloun-25-910-45
4,335,83475,335,in-mahameddeb,83475,pub-amira-samet-chabchoub-28-b2a-488
...,...,...,...,...,...,...
582,263828,451872,263828,pub-rÃ©my-vadrot-24-913-818,451872,pub-emmanuel-cabrinha-32-508-914
583,263828,385916,263828,pub-rÃ©my-vadrot-24-913-818,385916,pub-pierre-breton-1-b86-362
584,385916,451872,385916,pub-pierre-breton-1-b86-362,451872,pub-emmanuel-cabrinha-32-508-914
585,408921,469301,408921,pub-marco-pesce-6-688-750,469301,pub-carmelo-pistorio-4-779-aa6


In [46]:
edges_df = df2.filter(['source_username','target_username'], axis=1)
edges_df = edges_df.rename(columns={"source_username": "source", "target_username": "target"})
edges_df

,source,target
0,in-mahameddeb,pub-hatem-bouattour-17-7aa-882
1,in-mahameddeb,pub-anja-slama-16-52a-868
2,in-mahameddeb,pub-amal-hammami-46-170-5a3
3,in-mahameddeb,pub-anis-jabloun-25-910-45
4,in-mahameddeb,pub-amira-samet-chabchoub-28-b2a-488
...,...,...
582,pub-rÃ©my-vadrot-24-913-818,pub-emmanuel-cabrinha-32-508-914
583,pub-rÃ©my-vadrot-24-913-818,pub-pierre-breton-1-b86-362
584,pub-pierre-breton-1-b86-362,pub-emmanuel-cabrinha-32-508-914
585,pub-marco-pesce-6-688-750,pub-carmelo-pistorio-4-779-aa6


In [47]:
edges_df.to_excel("edges_df.xlsx")  

In [48]:
source_counts = edges_df['source'].value_counts().rename_axis('unique_values').reset_index(name='counts')
source_counts

,unique_values,counts
0,in-mahameddeb,103
1,pub-martin-bester-27-a0-612,39
2,pub-jenni-oakham-1a-220-b40,31
3,in-katdavid,30
4,in-yassineb,28
5,pub-francisco-galdos-anduaga-24-a39-912,28
6,in-alexandrelazar,28
7,in-simonefenili,25
8,pub-mohamed-nizar-ben-aba-1b-ba5-208,20
9,pub-alfredo-salgado-gerbi-11-672-50,20


In [49]:
data_subset = data_subset.merge(source_counts, left_on='username', right_on='unique_values', how='left')
data_subset = data_subset.drop(['unique_values'], axis=1)
data_subset = data_subset.fillna(0)
data_subset

,id,username,counts
0,62,pub-chiraz-ben-ali-28-583-525,1.0
1,116,pub-jules-gough-30-407-744,1.0
2,302,pub-hanen-abid-23-a7-b91,1.0
3,323,pub-amira-ben-ghanem-23-770-b03,1.0
4,335,in-mahameddeb,103.0
...,...,...,...
451,5512131,pub-kung-lagula-2b-7ab-3b6,0.0
452,5666442,pub-jim-copley-9-339-73b,0.0
453,5907069,pub-fabio-pasolini-16-b0a-96b,0.0
454,6088895,pub-clyde-burgess-1a-7a3-626,0.0


In [50]:
data_subset[data_subset.username.isin(['in-simonefenili'])]

,id,username,counts
26,23014,in-simonefenili,25.0


### Create 2 dictionaries nodes and edges and merge them into a json

In [51]:
nodes_df = data_subset.copy()
nodes_df = nodes_df.rename(columns={"id": "index", "username": "id"})
nodes_df = nodes_df.set_index('index')
nodes_df

,id,counts
index,,
62,pub-chiraz-ben-ali-28-583-525,1.0
116,pub-jules-gough-30-407-744,1.0
302,pub-hanen-abid-23-a7-b91,1.0
323,pub-amira-ben-ghanem-23-770-b03,1.0
335,in-mahameddeb,103.0
...,...,...
5512131,pub-kung-lagula-2b-7ab-3b6,0.0
5666442,pub-jim-copley-9-339-73b,0.0
5907069,pub-fabio-pasolini-16-b0a-96b,0.0


In [52]:
final_json = {}
jsdf = nodes_df.to_json(orient = "records")
final_json["nodes"] = json.loads(jsdf)

In [53]:
final_json

{'nodes': [{'id': 'pub-chiraz-ben-ali-28-583-525', 'counts': 1.0},
  {'id': 'pub-jules-gough-30-407-744', 'counts': 1.0},
  {'id': 'pub-hanen-abid-23-a7-b91', 'counts': 1.0},
  {'id': 'pub-amira-ben-ghanem-23-770-b03', 'counts': 1.0},
  {'id': 'in-mahameddeb', 'counts': 103.0},
  {'id': 'in-mariembelhadjkhelifa', 'counts': 0.0},
  {'id': 'pub-jihene-ben-amor-1b-514-219', 'counts': 0.0},
  {'id': 'pub-nesrine-gaidi-1a-318-903', 'counts': 0.0},
  {'id': 'pub-afef-chelbi-11-1b9-99', 'counts': 0.0},
  {'id': 'pub-emna-ben-chabchoubi-21-979-264', 'counts': 0.0},
  {'id': 'in-mariamhachem', 'counts': 0.0},
  {'id': 'pub-martin-bester-27-a0-612', 'counts': 39.0},
  {'id': 'pub-anja-slama-16-52a-868', 'counts': 0.0},
  {'id': 'pub-bigois-loic-10-482-17a', 'counts': 1.0},
  {'id': 'pub-wafa-laamiri-8-41a-3', 'counts': 0.0},
  {'id': 'pub-jenni-oakham-1a-220-b40', 'counts': 31.0},
  {'id': 'pub-carmelo-papa-b-6b4-aa7', 'counts': 2.0},
  {'id': 'pub-jaloul-ayed-0-579-394', 'counts': 1.0},
  {'id'

In [54]:
jsdf2 = edges_df.to_json(orient = "records")
final_json["links"] = json.loads(jsdf2)

In [55]:
final_json

{'nodes': [{'id': 'pub-chiraz-ben-ali-28-583-525', 'counts': 1.0},
  {'id': 'pub-jules-gough-30-407-744', 'counts': 1.0},
  {'id': 'pub-hanen-abid-23-a7-b91', 'counts': 1.0},
  {'id': 'pub-amira-ben-ghanem-23-770-b03', 'counts': 1.0},
  {'id': 'in-mahameddeb', 'counts': 103.0},
  {'id': 'in-mariembelhadjkhelifa', 'counts': 0.0},
  {'id': 'pub-jihene-ben-amor-1b-514-219', 'counts': 0.0},
  {'id': 'pub-nesrine-gaidi-1a-318-903', 'counts': 0.0},
  {'id': 'pub-afef-chelbi-11-1b9-99', 'counts': 0.0},
  {'id': 'pub-emna-ben-chabchoubi-21-979-264', 'counts': 0.0},
  {'id': 'in-mariamhachem', 'counts': 0.0},
  {'id': 'pub-martin-bester-27-a0-612', 'counts': 39.0},
  {'id': 'pub-anja-slama-16-52a-868', 'counts': 0.0},
  {'id': 'pub-bigois-loic-10-482-17a', 'counts': 1.0},
  {'id': 'pub-wafa-laamiri-8-41a-3', 'counts': 0.0},
  {'id': 'pub-jenni-oakham-1a-220-b40', 'counts': 31.0},
  {'id': 'pub-carmelo-papa-b-6b4-aa7', 'counts': 2.0},
  {'id': 'pub-jaloul-ayed-0-579-394', 'counts': 1.0},
  {'id'

In [56]:
jsonString = json.dumps(final_json)
jsonFile = open("data.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

In [40]:
fileObject = open("data.json", "r")
jsonContent = fileObject.read()
test = json.loads(jsonContent)
test

{'nodes': [{'id': 'in-mahameddeb', 'counts': 1},
  {'id': 'pub-martin-bester-27-a0-612', 'counts': 1},
  {'id': 'pub-jenni-oakham-1a-220-b40', 'counts': 1},
  {'id': 'in-yassineb', 'counts': 1},
  {'id': 'pub-francisco-galdos-anduaga-24-a39-912', 'counts': 1},
  {'id': 'in-katdavid', 'counts': 1},
  {'id': 'in-alexandrelazar', 'counts': 1},
  {'id': 'in-simonefenili', 'counts': 1},
  {'id': 'in-jcmonnet', 'counts': 20},
  {'id': 'pub-mohamed-nizar-ben-aba-1b-ba5-208', 'counts': 20},
  {'id': 'pub-sacha-martineau-10-36a-bb6', 'counts': 20},
  {'id': 'pub-alfredo-salgado-gerbi-11-672-50', 'counts': 20},
  {'id': 'pub-jayant-somani-0-453-544', 'counts': 20},
  {'id': 'pub-mags-vazquez-surtida-15-70-256', 'counts': 1},
  {'id': 'pub-mimie-del-rosario-b-a2b-835', 'counts': 1},
  {'id': 'pub-leena-bakre-8-33b-2b', 'counts': 1},
  {'id': 'pub-chantal-combi-1-669-79a', 'counts': 1},
  {'id': 'pub-rÃ©my-vadrot-24-913-818', 'counts': 1},
  {'id': 'in-chaybatac', 'counts': 10},
  {'id': 'pub-pier